In [ ]:
import numpy as np
import h5py

In [ ]:
hf = h5py.File('SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5', 'r')
hf1 = h5py.File('SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5', 'r')

In [ ]:
print(hf.keys())
print(hf1.keys())

<KeysViewHDF5 ['X', 'y']>
<KeysViewHDF5 ['X', 'y']>


In [ ]:
#For electrons
e_X = hf.get('X')
print(e_X)
e_y = hf.get('y')
print(e_y)

#for protons
p_X = hf1.get('X')
print(p_X)
p_y = hf1.get('y')
print(p_y)

<HDF5 dataset "X": shape (249000, 32, 32, 2), type "<f4">
<HDF5 dataset "y": shape (249000,), type "<f4">
<HDF5 dataset "X": shape (249000, 32, 32, 2), type "<f4">
<HDF5 dataset "y": shape (249000,), type "<f4">


In [ ]:
e_X = np.array(e_X)
print(e_X.shape)
e_y = np.array(e_y)
print(e_y.shape)

p_X = np.array(p_X)
print(p_X.shape)
p_y = np.array(p_y)
print(p_y.shape)

(249000, 32, 32, 2)
(249000,)
(249000, 32, 32, 2)
(249000,)


In [ ]:
hf.close()
hf1.close()

In [ ]:
e_train = []
p_train = []
e_val = []
p_val = []
e_test = []
p_test = []
for i in range(0,199200):
  e_train.append(e_X[i])
  p_train.append(p_X[i])

for j in range(199200, 224100):
  e_val.append(e_X[j])
  p_val.append(p_X[j])

for k in range(224100, 249000):
  e_test.append(e_X[k])
  p_test.append(p_X[k])

In [ ]:
X_train = np.concatenate([e_train, p_train])
X_val = np.concatenate([e_val, p_val])
X_test = np.concatenate([e_test, p_test])

In [ ]:
print((np.asarray(e_train)).shape)
print((np.asarray(p_train)).shape)
print((np.asarray(e_val)).shape)
print((np.asarray(p_val)).shape)
print((np.asarray(e_test)).shape)
print((np.asarray(p_test)).shape)

print((np.asarray(X_train)).shape)
print((np.asarray(X_val)).shape)
print((np.asarray(X_test)).shape)

(199200, 32, 32, 2)
(199200, 32, 32, 2)
(24900, 32, 32, 2)
(24900, 32, 32, 2)
(24900, 32, 32, 2)
(24900, 32, 32, 2)
(398400, 32, 32, 2)
(49800, 32, 32, 2)
(49800, 32, 32, 2)


In [ ]:
ey_train = []
py_train = []
ey_val = []
py_val = []
ey_test = []
py_test = []
for i in range(0,199200):
  ey_train.append(e_y[i])
  py_train.append(p_y[i])

for j in range(199200, 224100):
  ey_val.append(e_y[j])
  py_val.append(p_y[j])

for k in range(224100, 249000):
  ey_test.append(e_y[k])
  py_test.append(p_y[k])

In [ ]:
Y_train = np.concatenate([ey_train, py_train])
Y_val = np.concatenate([ey_val, py_val])
Y_test = np.concatenate([ey_test, py_test])

In [ ]:
print((np.asarray(ey_train)).shape)
print((np.asarray(py_train)).shape)
print((np.asarray(ey_val)).shape)
print((np.asarray(py_val)).shape)
print((np.asarray(ey_test)).shape)
print((np.asarray(py_test)).shape)

print((np.asarray(Y_train)).shape)
print((np.asarray(Y_val)).shape)
print((np.asarray(Y_test)).shape)

(199200,)
(199200,)
(24900,)
(24900,)
(24900,)
(24900,)
(398400,)
(49800,)
(49800,)


In [ ]:
import keras
from keras import backend as K
from keras.applications import InceptionV3
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization
import tensorflow as tf
from keras import layers
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, SGD

In [ ]:
ip=Input(shape=(32, 32, 2))
  
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block5d_conv4')(ip)
x=BatchNormalization(momentum=0.8)(x)
       
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3d_conv4')(x)
x=BatchNormalization(momentum=0.8)(x)
x=Dropout(.3)(x)  
        
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2d_conv2')(x)
x=BatchNormalization(momentum=0.8)(x)
        
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1d_conv2')(x)
x=BatchNormalization(momentum=0.8)(x)
x=Dropout(.5)(x)  

x=layers.Conv2D(3, (3, 3), activation='relu', padding='same')(x)
x=layers.Flatten()(x)
out=layers.Dense(1, activation="sigmoid")(x)
model=Model(ip,out)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 2)]       0         
_________________________________________________________________
block5d_conv4 (Conv2D)       (None, 32, 32, 256)       4864      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 256)       1024      
_________________________________________________________________
block3d_conv4 (Conv2D)       (None, 32, 32, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 256)       1024      
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 256)       0         
_________________________________________________________________
block2d_conv2 (Conv2D)       (None, 32, 32, 128)       295040

In [ ]:
opt=RMSprop(lr=0.0001)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.load_weights("t1_20.h5")
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=32,epochs=10,verbose=1)

Epoch 1/10
12450/12450 [==============================] - 834s 67ms/step - loss: 0.5614 - accuracy: 0.7205 - val_loss: 0.5659 - val_accuracy: 0.7164
Epoch 2/10
12450/12450 [==============================] - 832s 67ms/step - loss: 0.5606 - accuracy: 0.7217 - val_loss: 0.5638 - val_accuracy: 0.7184
Epoch 3/10
12450/12450 [==============================] - 831s 67ms/step - loss: 0.5598 - accuracy: 0.7221 - val_loss: 0.5605 - val_accuracy: 0.7184
Epoch 4/10
12450/12450 [==============================] - 832s 67ms/step - loss: 0.5593 - accuracy: 0.7224 - val_loss: 0.5600 - val_accuracy: 0.7171
Epoch 5/10
12450/12450 [==============================] - 829s 67ms/step - loss: 0.5587 - accuracy: 0.7233 - val_loss: 0.5596 - val_accuracy: 0.7172
Epoch 6/10
12450/12450 [==============================] - 823s 66ms/step - loss: 0.5581 - accuracy: 0.7236 - val_loss: 0.5571 - val_accuracy: 0.7214
Epoch 7/10
12450/12450 [==============================] - 828s 66ms/step - loss: 0.5576 - accuracy: 0.7241

In [ ]:
model.save_weights("t1_30.h5")

In [ ]:
model.load_weights("t1_30.h5")
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=32,epochs=20,verbose=1)

Epoch 1/20
12450/12450 [==============================] - 889s 69ms/step - loss: 0.5552 - accuracy: 0.7262 - val_loss: 0.5603 - val_accuracy: 0.7199
Epoch 2/20
12450/12450 [==============================] - 852s 68ms/step - loss: 0.5534 - accuracy: 0.7275 - val_loss: 0.5564 - val_accuracy: 0.7252
Epoch 3/20
12450/12450 [==============================] - 843s 68ms/step - loss: 0.5542 - accuracy: 0.7263 - val_loss: 0.5576 - val_accuracy: 0.7210
Epoch 4/20
12450/12450 [==============================] - 839s 67ms/step - loss: 0.5532 - accuracy: 0.7281 - val_loss: 0.5589 - val_accuracy: 0.7203
Epoch 5/20
12450/12450 [==============================] - 841s 68ms/step - loss: 0.5509 - accuracy: 0.7285 - val_loss: 0.5565 - val_accuracy: 0.7256
Epoch 6/20
12450/12450 [==============================] - 841s 68ms/step - loss: 0.5517 - accuracy: 0.7289 - val_loss: 0.5567 - val_accuracy: 0.7242
Epoch 7/20
12450/12450 [==============================] - 841s 68ms/step - loss: 0.5517 - accuracy: 0.7278

In [ ]:
model.save_weights("t1_50.h5")